EVENTUAL TITLE OF MANUSCRIPT
============================
This notebook is the computational appendix of [arXiv:1605.XXXXX](http://arxiv.abs/1605.XXXXX). We spell out in detail how to obtain the numerical results in the manuscript. We rely on [Trotter-Suzuki-MPI](https://trotter-suzuki-mpi.github.io/), a massively parallel solver for the Gross--Pitaevskii equation, and its [Python wrapper](http://trotter-suzuki-mpi.readthedocs.org/). The calculations use only a single computer, but it can take many hours to finish them.

Preliminaries
-------------
First we import the necessary modules and ensure that we get identical behaviour in Python 2 and 3.

In [ ]:
from __future__ import print_function, division
import trottersuzuki as ts
import numpy as np
import matplotlib.pyplot as plt

We will generate some files to save the results of the calculations. By default, they will go the current directory:

In [ ]:
directory = ""

Next we define a series of helper functions that we will use later. The following function returns the coordinates of a vortex position:

In [ ]:
def vortex_position(grid, state, approx_cloud_radius):
    delta_y = grid.length_y / float(grid.dim_y)
    delta_x = grid.length_x / float(grid.dim_x)
    matrix = state.get_phase()
    # calculate norm gradient matrix
    norm_grad = np.zeros((grid.dim_y, grid.dim_x))
    for idy in range(1, grid.dim_y-1):
        for idx in range(1, grid.dim_x-1):
            if (idx-grid.dim_x*0.5)**2 + (idy-grid.dim_y*0.5)**2 < \
                    approx_cloud_radius**2/delta_x**2:
                up = matrix[idy+1, idx]
                dw = matrix[idy-1, idx]
                rg = matrix[idy, idx+1]
                lf = matrix[idy, idx-1]

                if abs(up-dw) > np.pi:
                    up -= np.sign(up) * 2. * np.pi
                if abs(rg-lf) > np.pi:
                    rg -= np.sign(rg) * 2. * np.pi

                grad_x = (rg-lf)/(2.*delta_x)
                grad_y = (up-dw)/(2.*delta_y)
                norm_grad[idy, idx] = np.sqrt(grad_x**2 + grad_y**2)

    max_norm = np.nanmax(norm_grad)
    coord_x = []
    coord_y = []
    for idy in range(1, grid.dim_y-1):
        for idx in range(1, grid.dim_x-1):
            if norm_grad[idy, idx] >= max_norm*0.9:
                coord_x.append((idx + 0.5) * delta_x - 0.5 * grid.length_x)
                coord_y.append((idy + 0.5) * delta_y - 0.5 * grid.length_y)

    coords = np.zeros(2)
    for i in range(0, len(coord_x)):
        coords[1] += coord_y[i] / float(len(coord_x))
        coords[0] += coord_x[i] / float(len(coord_x))

    return coords


The following function plots heatmaps of the density and the phase:

In [ ]:
def heatmaps_of_density_and_phase(grid, state_a, state_b, hamiltonian,
                                  imag_time, mathCalL, rho, cont, time, export,
                                  show, x_pos_resc=None, y_pos_resc=None,
                                  x_pos_2_resc=None, y_pos_2_resc=None):

    if x_pos_resc is None:
        x_pos_resc = []
    if y_pos_resc is None:
        y_pos_resc = []
    if x_pos_2_resc is None:
        x_pos_2_resc = []
    if y_pos_2_resc is None:
        y_pos_2_resc = []

    x_ticks_1 = [0, grid.dim_x/4, grid.dim_x/2, 3*grid.dim_x/4, grid.dim_x]
    x_ticks_2 = [str(-grid.length_x/2), str(-grid.length_x/4), '0',
                 str(grid.length_x/4), str(grid.length_x/2)]
    density_A = state_a.get_particle_density()
    density_B = state_b.get_particle_density()
    phase_A = state_a.get_phase()
    phase_B = state_b.get_phase()
    plt.figure(figsize=(19, 4))

    title = ('$l_\\Omega/r_{12}=$ %4.3f,     $r_{12}/\\xi=$ %4.2f,     $g_{AB}/g=$ %4.2f,     $t/T_{\\rm Rabi}=$ %4.2f'
             % (mathCalL, rho, hamiltonian.coupling_ab/hamiltonian.coupling_a,
                (time*abs(hamiltonian.omega_r)/(2.*np.pi))))

    plt.suptitle(title, fontsize=16)

    data = [density_A, density_B, density_A-density_B, phase_A-phase_B]

    x_centers = [x_pos_resc, x_pos_2_resc, x_pos_resc, x_pos_2_resc]
    y_centers = [y_pos_resc, y_pos_2_resc, y_pos_resc, y_pos_2_resc]
    cmaps = ['afmhot', 'afmhot', 'hsv', 'hsv']
    for i in np.arange(len(data)):
        plt.subplot(1, 4, i+1)
        plt.xticks(x_ticks_1, x_ticks_2)
        plt.yticks(x_ticks_1, x_ticks_2)
        plt.pcolormesh(data[i], cmap=cmaps[i])
        plt.plot(x_centers[i], y_centers[i], linewidth=.5, color='green',
                 linestyle='solid', marker=None)
        if i < 2:
            color_indices = np.linspace(0, 10, len(x_centers[i]))
            plt.scatter(x_centers[i], y_centers[i], c=color_indices, cmap='Greys_r')
        else:
            color_indices = np.linspace(0, 10, len(x_centers[1]))
            plt.scatter(x_centers[1], y_centers[1], c=color_indices, cmap='Greys_r')
            color_indices = np.linspace(0, 10, len(x_centers[2]))
            plt.scatter(x_centers[2], y_centers[2], c=color_indices, cmap='Greys_r')

    if export:
        plt.savefig(directory + ('imagTime' if imag_time else 'realTime')
                    + '_plots_' + str(cont).zfill(3) + '.png')
    if not show:
        plt.ioff()
        plt.close()

Precession of two vortices in a two-component BEC
-------------------------------------------------
Details of what goes on in the function should come here.

In [ ]:
def two_vortices(lOmega_over_r12, r12_over_Xi12):
    ## Fixed variables
    length = 4.   # physical length of the lattice
    initial_vortices_distance = 1.  # physical distance between vortices cores
    n_0 = 4 / (np.pi * length**2)  # density at the center of the circular box

    ### Lattice parameters
    # Ensure that one coherence length will be equal to "const" spacings
    const = 3.
    dim = int(length * const * r12_over_Xi12)
    print('Linear dimension of the Lattice:', dim)

    ### Hamiltonian parameters
    g_a = r12_over_Xi12**2 / (2. * n_0)
    g_b = r12_over_Xi12**2 / (2. * n_0)
    g_ab = 0
    omega_rabi = -1. / (lOmega_over_r12**2)

    ### Solver parameters
    delta_t = 2.5e-5
    Rabi_period = 2 * np.pi/omega_rabi
    Rabi_periods_to_be_simulated = 0.5

    # Function of the external potential
    def external_potential(x, y):
        if (x**2 + y**2) >= (length/2.1)**2:
            return 1e10
        else:
            return 0.

    # Initial wave function
    def const_state(x, y):
        return 1./length

    # Define the vortex in component 1
    def vortex_a(x, y):
        z = (x-initial_vortices_distance/2.) + 1j*y
        angle = np.angle(z)
        return np.exp(1j * angle)

    # Define the vortex in component 2
    def vortex_b(x, y):
        z = (x+initial_vortices_distance/2.) + 1j*y
        angle = np.angle(z)
        return np.exp(1j * angle)

    # Set the geometry of the simulation
    grid = ts.Lattice(dim, length, length)
    # real time hamiltonian
    potential = ts.Potential(grid)
    potential.init_potential(external_potential)
    hamiltonian = ts.Hamiltonian2Component(grid, potential, potential, 1., 1.,
                                           g_a, g_ab, g_b, omega_rabi)

    state_a = ts.State(grid)  # Initialize the state in the first component
    state_b = ts.State(grid)  # Initialize the state in the second component
    state_a.init_state(const_state)
    state_b.init_state(const_state)

    # Print the vortexes
    state_a.imprint(vortex_a)
    state_b.imprint(vortex_b)

    # Initialize the solver
    solver = ts.Solver(grid, state_a, state_b, hamiltonian, delta_t)

    heatmaps_of_density_and_phase(grid, state_a, state_b, hamiltonian, True,
                                  lOmega_over_r12, r12_over_Xi12, 0, 0, True,
                                  False)

    iterations = 500
    coord_a = vortex_position(grid, state_a, length*0.45)
    coord_b = vortex_position(grid, state_b, length*0.45)
    vortex_distance = np.sqrt((coord_b[0]-coord_a[0])**2 +
                              (coord_b[1]-coord_a[1])**2)
    cont = 0

    for cont in range(6):
        solver.evolve(iterations, True)  # imaginary time evolution
        print(cont)
        time = (cont + 1) * iterations * delta_t
        coord_a = vortex_position(grid, state_a, length*0.45)
        coord_b = vortex_position(grid, state_b, length*0.45)
        vortex_distance = np.sqrt((coord_b[0]-coord_a[0])**2 +
                                  (coord_b[1]-coord_a[1])**2)
        heatmaps_of_density_and_phase(grid, state_a, state_b, hamiltonian,
                                      True, lOmega_over_r12, r12_over_Xi12,
                                      cont+1, time, True, False)
    print('---Imaginary evolution completed---')

    coord_a = vortex_position(grid, state_a, length*0.45)
    coord_b = vortex_position(grid, state_b, length*0.45)
    vortex_distance = np.sqrt((coord_b[0]-coord_a[0])**2 +
                              (coord_b[1]-coord_a[1])**2)

    out_file = open(str(lOmega_over_r12) + "_" + str(r12_over_Xi12), "w")
    out_file.write('Vortices distance before real time evolution: ' + str(vortex_distance) + "\n")

    print('Vortices distance before real time evolution:', vortex_distance)
    print('---Start real time evolution---')
    frames_per_Rabi_period = 20
    iterations = abs(int(Rabi_period / delta_t / frames_per_Rabi_period))
    max_it_real_time = int(frames_per_Rabi_period * Rabi_periods_to_be_simulated)
    print('Iterations between two snapshots:', iterations)
    print('Number of snapshots:', max_it_real_time)

    x_pos, y_pos = [], []
    x_pos_2, y_pos_2 = [], []
    radius_1, radius_2  = [], []
    angles_12 = []
    for cont in range(0, max_it_real_time):
        solver.evolve(iterations)
        coord_a = vortex_position(grid, state_a, length*0.45)
        coord_b = vortex_position(grid, state_b, length*0.45)
        time = (cont + 1) * iterations * delta_t
        print('Snapshot:', cont)

        radius_1.append(np.sqrt(coord_a[0]**2 + coord_a[1]**2))
        radius_2.append(np.sqrt(coord_b[0]**2 + coord_b[1]**2))
        angles_12.append(np.angle(coord_a[0] - coord_b[0] +
                                  1j * (coord_a[1] - coord_b[1])))

        # plot
        x_pos.append((coord_a[0] + length*0.5) / grid.delta_x)
        y_pos.append((coord_a[1] + length*0.5) / grid.delta_y)
        x_pos_2.append((coord_b[0] + length*0.5) / grid.delta_x)
        y_pos_2.append((coord_b[1] + length*0.5) / grid.delta_y)
        heatmaps_of_density_and_phase(grid, state_a, state_b, hamiltonian,
                                      False, lOmega_over_r12, r12_over_Xi12,
                                      cont, time, True, False, x_pos, y_pos,
                                      x_pos_2, y_pos_2)

    prec_freq_12 = np.zeros(len(x_pos)-1)
    for ind in range(0, len(x_pos)-1):
        if abs(angles_12[ind+1] - angles_12[ind]) > np.pi:
            angles_12[ind+1] -= np.sign(angles_12[ind+1]) * 2. * np.pi
        prec_freq_12[ind] = (angles_12[ind+1] - angles_12[ind]) / (iterations *
                                                                   delta_t)

    print('Mean freq of precession over rabi coupling:', np.mean(prec_freq_12) / omega_rabi)
    print('Expected result from eq(11) notes:', np.log(length / vortex_distance))
    print('Expected result from eq(12) notes:', 4.*np.sqrt(2.) / np.pi / np.sqrt(abs(omega_rabi)) / vortex_distance)

    out_file.write('Mean freq of precession over rabi coupling: ' + str(np.mean(prec_freq_12) / omega_rabi) + "\n")
    out_file.write('Theoretical eq(11) notes: ' + str(np.log(length / vortex_distance)) + "\n")
    out_file.write('Theoretical eq(12) notes: ' + str(4.*np.sqrt(2.) / np.pi / np.sqrt(abs(omega_rabi)) / vortex_distance) + "\n")
    out_file.close()

    plt.plot(radius_1)
    plt.xlabel('time')
    plt.ylabel('distance vortex 1 to lattice center')
    plt.savefig(str(lOmega_over_r12) + "_" + str(r12_over_Xi12) + "_radius1.png")
    plt.close()

    plt.plot(angles_12)
    plt.xlabel('time')
    plt.ylabel('angles between x-axis and the vector joining the center of two vortices')
    plt.savefig(str(lOmega_over_r12) + "_" + str(r12_over_Xi12) + "_angles12.png")
    plt.close()

    plt.plot(prec_freq_12)
    plt.xlabel('time')
    plt.ylabel('Precession frequency')
    plt.savefig(str(lOmega_over_r12) + "_" + str(r12_over_Xi12) + "_frequence_precession.png")
    plt.close()

We explore the parameter space:

In [ ]:
lOmega_over_r12 = 0.25
r12_over_Xi12 = 30.
two_vortices(lOmega_over_r12, r12_over_Xi12)

Transfer of a single vortex between a two-component BEC
-------------------------------------------------------
Gory details here.

Oscillations of the relative density
------------------------------------
Even more gory details here.